# 需用到的知识点

    1、Wavelet transform 小波变换  https://www.zhihu.com/question/22864189/answer/40772083
        Consequently, wavelet is useful in handling highly irregular financial time series
            https://www.researchgate.net/publication/223249583_Forecasting_stock_markets_using_wavelet_transforms_and_recurrent_neural_networks_An_integrated_system_based_on_artificial_bee_colony_algorithm
            
        https://pywavelets.readthedocs.io/en/latest/regression/index.html
        
        https://blog.csdn.net/zhaoyuxia517/article/details/78005713 基于小波变换的时间序列预测，Python实现
        
        http://www.docin.com/p-1546109075.html
        
        http://www.docin.com/p-1072414208.html?docfrom=rrela
        
        https://www.zhihu.com/question/19725983/answer/13856998
        
        http://www.360doc.com/content/13/0614/15/10724725_292827411.shtml
            
    2、Stacked autoencoders
        https://blog.csdn.net/jningwei/article/details/78836823
    
    另注意： 
    什么Autoencoder啦、RBM啦，现在都已经 没人用了 。
    现在所常说的 pre-training (预训练) ，其实 专指 migration learning (迁移学习)，那是一种无比强大又省事儿的trick。
    
        https://blog.csdn.net/hai008007/article/details/79676994

In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# 准备数据

In [89]:
# 包含部分指标数据
from stock.common.sdata import StockData
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
code = '600016'
sd = StockData(code)
data = sd.combine_income(ndays=5)
key_defines = ['RSI','MACD','ADOSC','OBV','MFI','BBANDS','BOP','ADX','EMA','MA']
# 指标数据

data = StockData.combine_index_data(data,index_keys=key_defines)

use_column = [x not in ['Flag','INCOME'] for x in data.columns.get_level_values(0)]
data.iloc[:,use_column] =  \
     data.iloc[:,use_column].apply(lambda x: (x-np.mean(x))/np.std(x))
target= data.loc[:,'Flag'].values

data_values = data.iloc[:,use_column].values
dlen = int(len(data_values)*0.8)
train_data = torch.from_numpy(np.array(data_values[0:dlen])).float()
train_target = target[0:dlen]

test_data = torch.from_numpy(np.array(data_values[dlen:])).float()
test_target = target[dlen:]
dataset = TensorDataset(train_data)
train_loader = DataLoader(dataset=dataset, batch_size=8, shuffle=True)

n_feature = len(data.columns) - 2
print(n_feature)

20


# Autoencode模型

In [90]:
# 自编码主要用于数据降维
import torch.nn as nn
class AutoEncoder(nn.Module):
    def __init__(self, n_feature):
        super(AutoEncoder,self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(n_feature, 16),
            nn.Tanh(),
            nn.Linear(16, 8),
            nn.Tanh(),
            nn.Linear(8, 3)
        )
        self.decoder = nn.Sequential(
            nn.Linear(3,8),
            nn.Tanh(),
            nn.Linear(8,16),
            nn.Tanh(),            
            nn.Linear(16, n_feature)
        )
    def forward(self,x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded        
    
   

# 模型训练

In [98]:
import torch
import os
import copy
LR = 0.0001
autoencoder = AutoEncoder(n_feature = n_feature)
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=LR)
loss_func = nn.MSELoss()
epcho = 50000

if os.path.exists('/home/hecong/dplearn/model/autoencode_model.pkl'):
    print('加载上次训练模型...')
    autoencoder.load_state_dict(torch.load('/home/hecong/dplearn/model/autoencode_model.pkl'))
    
for epcho in range(epcho):
    run_loss = 0
    for step, x in enumerate(train_loader):
#         print(x[0])
#         print(x[0].shape)
        in_x = x[0]
        target  = x[0]
        encode,decode = autoencoder(in_x)
        loss = loss_func(decode, target)
        run_loss = run_loss + loss
        optimizer.zero_grad()               # clear gradients for this training step
        loss.backward()                     # backpropagation, compute gradients
        optimizer.step()                    # apply gradients        
        
    if epcho % 5 == 0:
        print('epcho --> {} ： loss {:.3f}'.format(epcho, run_loss/step))
        _wts = copy.deepcopy(autoencoder.state_dict())  
        torch.save(_wts,'/home/hecong/dplearn/model/autoencode_model.pkl')            
        

加载上次训练模型...
epcho --> 0 ： loss 0.088
epcho --> 5 ： loss 0.088
epcho --> 10 ： loss 0.088
epcho --> 15 ： loss 0.089
epcho --> 20 ： loss 0.088
epcho --> 25 ： loss 0.089
epcho --> 30 ： loss 0.088
epcho --> 35 ： loss 0.089
epcho --> 40 ： loss 0.088
epcho --> 45 ： loss 0.089
epcho --> 50 ： loss 0.088
epcho --> 55 ： loss 0.088
epcho --> 60 ： loss 0.088
epcho --> 65 ： loss 0.088
epcho --> 70 ： loss 0.088
epcho --> 75 ： loss 0.088
epcho --> 80 ： loss 0.089
epcho --> 85 ： loss 0.088
epcho --> 90 ： loss 0.088
epcho --> 95 ： loss 0.088
epcho --> 100 ： loss 0.088
epcho --> 105 ： loss 0.088
epcho --> 110 ： loss 0.088
epcho --> 115 ： loss 0.088
epcho --> 120 ： loss 0.088
epcho --> 125 ： loss 0.088
epcho --> 130 ： loss 0.088
epcho --> 135 ： loss 0.088
epcho --> 140 ： loss 0.088
epcho --> 145 ： loss 0.088
epcho --> 150 ： loss 0.089
epcho --> 155 ： loss 0.088
epcho --> 160 ： loss 0.089
epcho --> 165 ： loss 0.088
epcho --> 170 ： loss 0.088
epcho --> 175 ： loss 0.088
epcho --> 180 ： loss 0.088
epcho --> 1

epcho --> 1505 ： loss 0.088
epcho --> 1510 ： loss 0.088
epcho --> 1515 ： loss 0.088
epcho --> 1520 ： loss 0.088
epcho --> 1525 ： loss 0.088
epcho --> 1530 ： loss 0.088
epcho --> 1535 ： loss 0.088
epcho --> 1540 ： loss 0.088
epcho --> 1545 ： loss 0.088
epcho --> 1550 ： loss 0.088
epcho --> 1555 ： loss 0.088
epcho --> 1560 ： loss 0.088
epcho --> 1565 ： loss 0.088
epcho --> 1570 ： loss 0.088
epcho --> 1575 ： loss 0.088
epcho --> 1580 ： loss 0.088
epcho --> 1585 ： loss 0.088
epcho --> 1590 ： loss 0.088
epcho --> 1595 ： loss 0.088
epcho --> 1600 ： loss 0.088
epcho --> 1605 ： loss 0.088
epcho --> 1610 ： loss 0.088
epcho --> 1615 ： loss 0.088
epcho --> 1620 ： loss 0.088
epcho --> 1625 ： loss 0.088
epcho --> 1630 ： loss 0.088
epcho --> 1635 ： loss 0.088
epcho --> 1640 ： loss 0.088
epcho --> 1645 ： loss 0.088
epcho --> 1650 ： loss 0.088
epcho --> 1655 ： loss 0.088
epcho --> 1660 ： loss 0.088
epcho --> 1665 ： loss 0.088
epcho --> 1670 ： loss 0.088
epcho --> 1675 ： loss 0.088
epcho --> 1680 ： los

epcho --> 2970 ： loss 0.088
epcho --> 2975 ： loss 0.088
epcho --> 2980 ： loss 0.088
epcho --> 2985 ： loss 0.088
epcho --> 2990 ： loss 0.088
epcho --> 2995 ： loss 0.088
epcho --> 3000 ： loss 0.088
epcho --> 3005 ： loss 0.088
epcho --> 3010 ： loss 0.088
epcho --> 3015 ： loss 0.088
epcho --> 3020 ： loss 0.088
epcho --> 3025 ： loss 0.088
epcho --> 3030 ： loss 0.088
epcho --> 3035 ： loss 0.088
epcho --> 3040 ： loss 0.088
epcho --> 3045 ： loss 0.088
epcho --> 3050 ： loss 0.088
epcho --> 3055 ： loss 0.088
epcho --> 3060 ： loss 0.088
epcho --> 3065 ： loss 0.088
epcho --> 3070 ： loss 0.087
epcho --> 3075 ： loss 0.088
epcho --> 3080 ： loss 0.088
epcho --> 3085 ： loss 0.088
epcho --> 3090 ： loss 0.088
epcho --> 3095 ： loss 0.088
epcho --> 3100 ： loss 0.088
epcho --> 3105 ： loss 0.088
epcho --> 3110 ： loss 0.088
epcho --> 3115 ： loss 0.088
epcho --> 3120 ： loss 0.087
epcho --> 3125 ： loss 0.087
epcho --> 3130 ： loss 0.088
epcho --> 3135 ： loss 0.088
epcho --> 3140 ： loss 0.088
epcho --> 3145 ： los

KeyboardInterrupt: 

In [99]:
autoencoder = AutoEncoder(n_feature = n_feature)
autoencoder.load_state_dict(torch.load('/home/hecong/dplearn/model/autoencode_model.pkl'))
encode, decode = autoencoder(test_data)
print(decode)
print(test_data)

tensor([[ 0.2963,  0.2585,  0.2607,  ..., -0.5424,  0.3590,  0.3663],
        [ 0.2741,  0.2462,  0.2562,  ..., -0.5206,  0.3639,  0.3743],
        [ 0.1847,  0.1893,  0.2196,  ..., -0.2821,  0.3042,  0.3137],
        ...,
        [-0.0709, -0.0973, -0.1053,  ..., -0.2487,  0.1034,  0.1332],
        [-0.0457, -0.0441, -0.0236,  ..., -0.1772,  0.1133,  0.1354],
        [-0.0962, -0.0890, -0.0652,  ..., -0.1568,  0.0548,  0.0758]],
       grad_fn=<ThAddmmBackward>)
tensor([[ 0.1551,  0.1076,  0.1283,  ..., -1.1425,  0.2122,  0.2478],
        [ 0.1354,  0.0999,  0.1165,  ..., -1.1969,  0.2066,  0.2351],
        [ 0.0841,  0.1038,  0.1283,  ..., -1.1982,  0.2022,  0.2220],
        ...,
        [-0.1642, -0.1878, -0.2069,  ..., -0.5384, -0.0984, -0.1132],
        [-0.1878, -0.2109, -0.1871,  ..., -0.4280, -0.1039, -0.1156],
        [-0.1760, -0.1993, -0.1674,  ..., -0.3667, -0.1078, -0.1183]])


# 分类数据准备

In [100]:
import torch
autoencoder = AutoEncoder(n_feature = n_feature)
autoencoder.load_state_dict(torch.load('/home/hecong/dplearn/model/autoencode_model.pkl'))
data_encode, _ = autoencoder(train_data)
data_encode = data_encode.detach().numpy()
test_encode, _ = autoencoder(test_data)
test_encode = test_encode.detach().numpy()



# KNN 分类

In [101]:
from sklearn import neighbors
knn = neighbors.KNeighborsClassifier() 
fit = knn.fit(data_encode,train_target)
print(knn.score(test_encode,test_target))
result = knn.predict(test_encode)
print(len(result[result==test_target])/len(result))

0.479646017699115
0.479646017699115


# SVM 分类

In [102]:
test_idx, train_idx = np.where(test_target==1), np.where(train_target == 1)
test_target[test_idx]=0
train_target[train_idx]=0

test_idx, train_idx = np.where(test_target==2),  np.where(train_target==2)
test_target[test_idx] = 1
train_target[train_idx] = 1

In [103]:
# http://scikit-learn.org/stable/modules/classes.html

from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV   # sklearn 调参 采用  grid search 方式 
clf = SVC(gamma='auto')
param_grid = {'C': [0.01,0.1,0.5, 1, 10], 'gamma': ['auto'],'kernel':['rbf','poly','sigmoid','linear']}
grid_search = GridSearchCV(clf, param_grid, n_jobs = 1, verbose=1)  
grid_search.fit(data_encode, train_target)  
best_parameters = grid_search.best_estimator_.get_params()  
for para, val in best_parameters.items():  
    print(para, val)  
    
model = SVC(C=best_parameters['C'], gamma=best_parameters['gamma'], probability=False,kernel=best_parameters['kernel'])      
model.fit(data_encode,train_target)
clf.fit(data_encode,train_target)  
print(clf)
print(model)
print(clf.score(test_encode,test_target))
print(model.score(test_encode,test_target))

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    6.9s finished


C 0.01
cache_size 200
class_weight None
coef0 0.0
decision_function_shape ovr
degree 3
gamma auto
kernel rbf
max_iter -1
probability False
random_state None
shrinking True
tol 0.001
verbose False
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
SVC(C=0.01, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
0.6442477876106195
0.35575221238938054
